In [63]:
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.clustering.KMeans
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.abs
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.{min, max, lit}
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{ IntegerType, StructField, StructType, StringType, DoubleType}
val sqlContext = new SQLContext(sc)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.clustering.KMeans
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.abs
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.{min, max, lit}
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{IntegerType, StructField, StructType, StringType, DoubleType}
sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@4934a240


In [64]:
val schema = new StructType().
add("artist_familiarity",DoubleType,true).
add("artist_hotttnesss",DoubleType,true).
add("artist_id",StringType,true).
add("artist_location",StringType,true).
add("artist_mbtags",StringType,true).
add("artist_mbtags_count",StringType,true).
add("artist_name",StringType,true).
add("artist_terms",StringType,true).
add("artist_terms_freq",StringType,true).
add("artist_terms_weight",StringType,true).
add("danceability",DoubleType,true).
add("duration",DoubleType,true).
add("end_of_fade_in",DoubleType,true).
add("energy",DoubleType,true).
add("key",DoubleType,true).
add("key_confidence",DoubleType,true).
add("loudness",DoubleType,true).
add("mode",DoubleType,true).
add("mode_confidence",DoubleType,true).
add("release",StringType,true).
add("segments_confidence",StringType,true).
add("segments_loudness_max",StringType,true).
add("segments_loudness_max_time",StringType,true).
add("segments_pitches",StringType,true).
add("segments_timbre",StringType,true).
add("similar_artists",StringType,true).
add("song_hotttnesss",DoubleType,true).
add("song_id",StringType,true).
add("start_of_fade_out",DoubleType,true).
add("tempo",DoubleType,true).
add("time_signature",IntegerType,true).
add("time_signature_confidence",DoubleType,true).
add("title",StringType,true).
add("track_id",StringType,true).
add("year",IntegerType,true)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

schema: org.apache.spark.sql.types.StructType = StructType(StructField(artist_familiarity,DoubleType,true), StructField(artist_hotttnesss,DoubleType,true), StructField(artist_id,StringType,true), StructField(artist_location,StringType,true), StructField(artist_mbtags,StringType,true), StructField(artist_mbtags_count,StringType,true), StructField(artist_name,StringType,true), StructField(artist_terms,StringType,true), StructField(artist_terms_freq,StringType,true), StructField(artist_terms_weight,StringType,true), StructField(danceability,DoubleType,true), StructField(duration,DoubleType,true), StructField(end_of_fade_in,DoubleType,true), StructField(energy,DoubleType,true), StructField(key,DoubleType,true), StructField(key_confidence,DoubleType,true), StructField(loudness,DoubleType,tru...

In [65]:
val msdf = sqlContext.read.format("com.databricks.spark.csv").
option("header", "false").
option("delimiter", "\t").
option("inferSchema", "false").
schema(schema).load("s3://millionsongprocessed/data_A.tsv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

msdf: org.apache.spark.sql.DataFrame = [artist_familiarity: double, artist_hotttnesss: double ... 33 more fields]


In [151]:


val usdf= sqlContext.read.format("com.databricks.spark.csv").
option("header","false").
option("delimiter", "\t").
load("s3://millionsongprocessed/train_triplets.txt")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

usdf: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 1 more field]


In [152]:
usdf.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+---+
|                 _c0|               _c1|_c2|
+--------------------+------------------+---+
|b80344d063b5ccb32...|SOAKIMP12A8C130995|  1|
|b80344d063b5ccb32...|SOAPDEY12A81C210A9|  1|
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|  2|
|b80344d063b5ccb32...|SOBFNSP12AF72A0E22|  1|
|b80344d063b5ccb32...|SOBFOVM12A58A7D494|  1|
|b80344d063b5ccb32...|SOBNZDC12A6D4FC103|  1|
|b80344d063b5ccb32...|SOBSUJE12A6D4F8CF5|  2|
|b80344d063b5ccb32...|SOBVFZR12A6D4F8AE3|  1|
|b80344d063b5ccb32...|SOBXALG12A8C13C108|  1|
|b80344d063b5ccb32...|SOBXHDL12A81C204C0|  1|
+--------------------+------------------+---+
only showing top 10 rows



In [66]:
msdf.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------------------+-----------------+-------------+-------------------+--------------------+--------------------+--------------------+--------------------+------------+---------+--------------+------+---+--------------+--------+----+---------------+--------------------+--------------------+---------------------+--------------------------+--------------------+--------------------+--------------------+-------------------+------------------+-----------------+-------+--------------+-------------------------+--------------------+------------------+----+
|  artist_familiarity|  artist_hotttnesss|         artist_id|  artist_location|artist_mbtags|artist_mbtags_count|         artist_name|        artist_terms|   artist_terms_freq| artist_terms_weight|danceability| duration|end_of_fade_in|energy|key|key_confidence|loudness|mode|mode_confidence|             release| segments_confidence|segments_loudness_max|segments_loudness_max_time|    segments_pitches|

In [67]:
var msdf2 = msdf.withColumn("time_signature", when(col("time_signature_confidence")<0.5, 4).otherwise(col("time_signature")))
val msdf3 = msdf2.select($"duration", $"loudness", $"artist_hotttnesss", $"tempo"/$"time_signature" as "speed", $"song_id")
var msdf4 = msdf3.na.drop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

msdf2: org.apache.spark.sql.DataFrame = [artist_familiarity: double, artist_hotttnesss: double ... 33 more fields]
msdf3: org.apache.spark.sql.DataFrame = [duration: double, loudness: double ... 3 more fields]
msdf4: org.apache.spark.sql.DataFrame = [duration: double, loudness: double ... 3 more fields]


In [68]:
msdf4.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------+-------------------+--------+------------------+
| duration|loudness|  artist_hotttnesss|   speed|           song_id|
+---------+--------+-------------------+--------+------------------+
| 213.9424|  -4.769|0.46131833754118956|  24.986|SOBLFFE12AF72AA5BA|
|118.07302|   -7.24|0.38660636406584575|31.36875|SOQPWCR12A6D4FB2A3|
|218.93179| -11.197| 0.4019975433642836| 23.0495|SOMZWCG12A8C13C480|
|193.01832| -13.179|0.30708016881260153|10.31975|SOJDASC12A8C13EB49|
|252.94322|  -7.227|0.37928237433191514| 40.4975|SOFRDWL12A58A7CEF7|
|222.35383|  -5.232| 0.3382898159400986|  25.203|SOZIULX12A8AE46C39|
|396.35546| -13.225|  0.387616855902411|  34.192|SOCSNVI12A8C13ECC2|
|177.47546|  -9.689|0.34342837829688244| 25.0175|SOXVLOJ12AB0189215|
|129.85424|  -9.495|                0.0|39.16925|SOEKAZG12AB018837E|
| 246.9873|  -8.849| 0.5620614073671926|  38.039|SOXZYWX12A6310ED0C|
+---------+--------+-------------------+--------+------------------+
only showing top 10 rows



In [69]:
//feature scaling
val scaledRange = lit(4)
val scaledMin = lit(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

scaledRange: org.apache.spark.sql.Column = 4
scaledMin: org.apache.spark.sql.Column = 1


In [135]:
@transient val (dMin, dMax) = msdf4.agg(min($"duration"), max($"duration")).first match { case Row(x: Double, y: Double) => (x,y)}
@transient val dNorm = ($"duration"-dMin)/(dMax-dMin)
@transient val dScal = scaledRange*dNorm + scaledMin

@transient val (lMin, lMax) = msdf4.agg(min($"loudness"), max($"loudness")).first match { case Row(x: Double, y: Double) => (x,y)}
@transient val lNorm = ($"loudness"-lMin)/(lMax-lMin)
@transient val lScal = scaledRange*lNorm + scaledMin

@transient val (aMin, aMax) = msdf4.agg(min($"artist_hotttnesss"), max($"artist_hotttnesss")).first match { case Row(x: Double, y: Double) => (x,y)}
@transient val aNorm = ($"artist_hotttnesss"-aMin)/(aMax-aMin)
@transient val aScal = scaledRange*aNorm + scaledMin

@transient val (sMin, sMax) = msdf4.agg(min($"speed"), max($"speed")).first match { case Row(x: Double, y: Double) => (x,y)}
@transient val sNorm = ($"speed"-sMin)/(sMax-sMin)
@transient val sScal = scaledRange*sNorm + scaledMin

val msdf5 = msdf4.withColumn("dScal",dScal).withColumn("lScal", lScal).withColumn("aScal", aScal).withColumn("sScal", sScal)
val allDF = msdf5.select("dScal", "lScal", "aScal", "sScal", "song_id")

// val rowsRDD = allDF.rdd.map(r => (r.getDouble(0), r.getDouble(1), r.getDouble(2), r.getDouble(3), r.getString(4)))
// rowsRDD.cache()
val vectors = allDF.rdd.map(r => Vectors.dense(r.getDouble(0), r.getDouble(1), r.getDouble(2), r.getDouble(3)))
vectors.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dMin: Double = 0.54812
dMax: Double = 3007.73832
dNorm: org.apache.spark.sql.Column = ((duration - 0.54812) / 3007.1902)
dScal: org.apache.spark.sql.Column = ((4 * ((duration - 0.54812) / 3007.1902)) + 1)
lMin: Double = -56.145
lMax: Double = 3.634
lNorm: org.apache.spark.sql.Column = ((loudness - -56.145) / 59.779)
lScal: org.apache.spark.sql.Column = ((4 * ((loudness - -56.145) / 59.779)) + 1)
aMin: Double = 0.0
aMax: Double = 0.8521862384017338
aNorm: org.apache.spark.sql.Column = ((artist_hotttnesss - 0.0) / 0.8521862384017338)
aScal: org.apache.spark.sql.Column = ((4 * ((artist_hotttnesss - 0.0) / 0.8521862384017338)) + 1)
sMin: Double = 0.0
sMax: Double = 191.267
sNorm: org.apache.spark.sql.Column = ((speed - 0.0) / 191.267)
sScal: org.apache.spark.sql.Column = ((4 * ((speed - 0.0) / 191.267)) + 1)
msdf5: org.apache.spark.sql.DataFrame = [duration: double, loudness: double ... 7 more fields]
allDF: org.apache.spark.sql.DataFrame = [dScal: double, lScal: double ... 3 more fields]


In [75]:
rowsRDD.take(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res166: Array[(Double, Double, Double, Double, String)] = Array((1.2838454049231738,4.437728968366818,3.1653404702070156,1.5225365588418285,SOBLFFE12AF72AA5BA), (1.1563251968565207,4.272386624065307,2.8146566872092276,1.6560201184731291,SOQPWCR12A6D4FB2A3), (1.2904820187296433,4.007611368540792,2.8868999533164286,1.4820381979118196,SOMZWCG12A8C13C480), (1.2560133376332498,3.874989544823433,2.4413758635132483,1.2158187246101,SOJDASC12A8C13EB49), (1.3357221635000007,4.2732564947556835,2.7802792734285653,1.846931253169653,SOFRDWL12A58A7CEF7), (1.2950338292536334,4.406748189163419,2.5878680067613273,1.5270747175414474,SOZIULX12A8AE46C39), (1.5264812847554503,3.8719115408421017,2.819399743555504,1.715063236209069,SOCSNVI12A8C13ECC2), (1.2353390749943252,4.108516368624434,2.611987440402599,1....

In [101]:
vectors.take(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res209: Array[(String, org.apache.spark.mllib.linalg.Vector)] = Array((SOBLFFE12AF72AA5BA,[1.2838454049231738,4.437728968366818,3.1653404702070156,1.5225365588418285]), (SOQPWCR12A6D4FB2A3,[1.1563251968565207,4.272386624065307,2.8146566872092276,1.6560201184731291]), (SOMZWCG12A8C13C480,[1.2904820187296433,4.007611368540792,2.8868999533164286,1.4820381979118196]), (SOJDASC12A8C13EB49,[1.2560133376332498,3.874989544823433,2.4413758635132483,1.2158187246101]), (SOFRDWL12A58A7CEF7,[1.3357221635000007,4.2732564947556835,2.7802792734285653,1.846931253169653]), (SOZIULX12A8AE46C39,[1.2950338292536334,4.406748189163419,2.5878680067613273,1.5270747175414474]), (SOCSNVI12A8C13ECC2,[1.5264812847554503,3.8719115408421017,2.819399743555504,1.715063236209069]), (SOXVLOJ12AB0189215,[1.235339074994325...

In [ ]:
// for(k <- 5 to 10)
// {
//     val kMeansModel = KMeans.train(vectors, k, 20)
//     val WSSE = kMeansModel.computeCost(vectors)
//     println("K = " + k)
//     println("Within set SSE = " + WSSE)
// }

In [86]:
val kMeansModel = KMeans.train(vectors, 7, 20)
kMeansModel.clusterCenters.foreach(println)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

kMeansModel: org.apache.spark.mllib.clustering.KMeansModel = org.apache.spark.mllib.clustering.KMeansModel@122678ac
[1.3313376811150552,4.258193489003848,3.3171621382761014,1.6681230377827574]
[1.3372745655678149,4.032418346807902,1.030419643781348,1.650965592302267]
[1.3312042999324307,4.320042676672932,2.7375657291670548,1.6854935257840982]
[1.3294464129241215,3.8892302755050885,2.790777422189137,1.6187933974820314]
[1.3117883089918125,3.27239088521847,2.6335767010173,1.563434633120673]
[1.326832216251905,4.04026153653496,2.6720023079692563,2.7697545977506572]
[1.3370129159516557,4.105945850519583,2.2567225545033445,1.650451872796106]


In [91]:
val WSSE = kMeansModel.computeCost(vectors)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

WSSE: Double = 3683.7879220247282


In [112]:
val predictions = kMeansModel.predict(vectors)
// val predictions = allDF.map{r => (r.getString(4), kMeansModel.predict(Vectors.dense(r.getDouble(0), r.getDouble(1), r.getDouble(2), r.getDouble(3))))}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[700] at map at KMeansModel.scala:85


In [113]:
predictions.take(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res230: Array[Int] = Array(0, 2)


In [161]:
val clusters = predictions.toDF("Cluster")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

clusters: org.apache.spark.sql.DataFrame = [Cluster: int]


In [163]:
clusters.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|Cluster|
+-------+
|      0|
|      2|
|      3|
|      6|
|      2|
|      2|
|      3|
|      2|
|      1|
|      0|
|      3|
|      5|
|      2|
|      6|
|      2|
|      1|
|      1|
|      3|
|      0|
|      6|
+-------+
only showing top 20 rows



In [164]:
val df1 = allDF.withColumn("df1Key", monotonically_increasing_id())
val df2 = songclusters.withColumn("df2Key", monotonically_increasing_id())
val songclusters = df2.join(df1, $"df1Key" === $"df2Key").drop("df1Key", "df2Key", "dScal", "lScal", "aScal", "sScal")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1: org.apache.spark.sql.DataFrame = [dScal: double, lScal: double ... 4 more fields]
df2: org.apache.spark.sql.DataFrame = [Cluster: int, df2Key: bigint]
songclusters: org.apache.spark.sql.DataFrame = [Cluster: int, song_id: string]


In [165]:
songclusters.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|Cluster|           song_id|
+-------+------------------+
|      2|SOYSPCB12A8C141C86|
|      2|SOMFRKT12A8C146C67|
|      2|SONWQQX12AC3DF6762|
|      2|SOHPVWT12AB0184E54|
|      2|SOZRRMP12A58A75BEB|
|      6|SONBSMJ12AC4687AED|
|      2|SOXWMCD12AB01810DB|
|      3|SOVBANJ12AAFF444CF|
|      2|SOEAXKZ12A81C208DA|
|      3|SOAYOUE12AB01888A6|
|      2|SOEHOJX12AB018394D|
|      3|SOBLTSM12A6D4F9EEA|
|      2|SOIIZSD12AB0184A86|
|      0|SOYOJKY12AB0185DFD|
|      3|SONWUSK12AB01837FD|
|      3|SOYHTLP12A6D4FC356|
|      2|SOKXFFC12A8C133922|
|      3|SOGIKFN12AC3DF7E39|
|      0|SOGAJHS12A8C13C37B|
|      2|SOLMIFQ12AB0183BB2|
+-------+------------------+
only showing top 20 rows



In [182]:
val list = List("user_id", "SongId", "PlayCount")
import spark.implicits._
val usdf2 = usdf.toDF(list:_*)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

list: List[String] = List(user_id, SongId, PlayCount)
import spark.implicits._
usdf2: org.apache.spark.sql.DataFrame = [user_id: string, SongId: string ... 1 more field]


In [183]:
usdf2.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+---------+
|             user_id|            SongId|PlayCount|
+--------------------+------------------+---------+
|b80344d063b5ccb32...|SOAKIMP12A8C130995|        1|
|b80344d063b5ccb32...|SOAPDEY12A81C210A9|        1|
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|        2|
|b80344d063b5ccb32...|SOBFNSP12AF72A0E22|        1|
|b80344d063b5ccb32...|SOBFOVM12A58A7D494|        1|
|b80344d063b5ccb32...|SOBNZDC12A6D4FC103|        1|
|b80344d063b5ccb32...|SOBSUJE12A6D4F8CF5|        2|
|b80344d063b5ccb32...|SOBVFZR12A6D4F8AE3|        1|
|b80344d063b5ccb32...|SOBXALG12A8C13C108|        1|
|b80344d063b5ccb32...|SOBXHDL12A81C204C0|        1|
+--------------------+------------------+---------+
only showing top 10 rows



In [184]:
val userclusters = usdf2.join(songclusters, usdf2("SongId")===clusterDF("song_id"))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

userclusters: org.apache.spark.sql.DataFrame = [user_id: string, SongId: string ... 3 more fields]


In [179]:
userclusters.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+---------+-------+------------------+
|             user_id|           Song_id|PlayCount|Cluster|           song_id|
+--------------------+------------------+---------+-------+------------------+
|2a0bfd858b2ed706d...|SOTSIIH12A8C13A516|        1|      2|SOTSIIH12A8C13A516|
+--------------------+------------------+---------+-------+------------------+
only showing top 1 row



In [185]:
val msdf6 = msdf4.withColumnRenamed("song_id","songID")
msdf6.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

msdf6: org.apache.spark.sql.DataFrame = [duration: double, loudness: double ... 3 more fields]
+---------+--------+-------------------+--------+------------------+
| duration|loudness|  artist_hotttnesss|   speed|            songID|
+---------+--------+-------------------+--------+------------------+
| 213.9424|  -4.769|0.46131833754118956|  24.986|SOBLFFE12AF72AA5BA|
|118.07302|   -7.24|0.38660636406584575|31.36875|SOQPWCR12A6D4FB2A3|
|218.93179| -11.197| 0.4019975433642836| 23.0495|SOMZWCG12A8C13C480|
|193.01832| -13.179|0.30708016881260153|10.31975|SOJDASC12A8C13EB49|
|252.94322|  -7.227|0.37928237433191514| 40.4975|SOFRDWL12A58A7CEF7|
|222.35383|  -5.232| 0.3382898159400986|  25.203|SOZIULX12A8AE46C39|
|396.35546| -13.225|  0.387616855902411|  34.192|SOCSNVI12A8C13ECC2|
|177.47546|  -9.689|0.34342837829688244| 25.0175|SOXVLOJ12AB0189215|
|129.85424|  -9.495|                0.0|39.16925|SOEKAZG12AB018837E|
| 246.9873|  -8.849| 0.5620614073671926|  38.039|SOXZYWX12A6310ED0C|
+-------

In [187]:
val topfeats = userclusters.join(msdf6, userclusters("SongId")===msdf6("songID")).orderBy(desc("PlayCount")).select("Cluster","duration","loudness","artist_hotttnesss","speed")
topfeats.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

topfeats: org.apache.spark.sql.DataFrame = [Cluster: int, duration: double ... 3 more fields]
+-------+---------+--------+-------------------+--------+
|Cluster| duration|loudness|  artist_hotttnesss|   speed|
+-------+---------+--------+-------------------+--------+
|      3|340.94975|  -6.166| 0.4503437220921902| 45.1225|
|      3|112.79628| -15.889|0.45514504003079936| 32.4805|
|      2|246.38649|  -6.924| 0.3694193175081429|  24.045|
|      0|189.98812|  -5.332| 0.5028240318123973|  41.741|
|      0|226.16771|  -4.857| 0.5223726680916047|  32.507|
|      0|237.76608|  -4.916|  0.796313707811507|24.76925|
|      3| 230.5824| -14.572|  0.352224614510411| 31.4925|
|      2|244.87138|  -5.609|0.37546522455148185|  38.898|
|      0|237.76608|  -4.916|  0.796313707811507|24.76925|
|      2| 203.2322|  -4.703|0.33536857774641954| 26.1775|
|      2|207.20281|  -5.366|0.41946463710949256|  24.269|
|      1|101.19791|  -5.623|                0.0|43.60225|
|      2|262.37342|   -9.37|0.352338

In [188]:
val tf = topfeats.head(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tf: Array[org.apache.spark.sql.Row] = Array([0,189.98812,-5.332,0.5028240318123973,41.741], [2,244.87138,-5.609,0.37546522455148185,38.898], [3,230.5824,-14.572,0.352224614510411,31.4925])


In [189]:
val song1 = tf(0)
val song2 = tf(1)
val song3 = tf(2)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

song1: org.apache.spark.sql.Row = [0,189.98812,-5.332,0.5028240318123973,41.741]
song2: org.apache.spark.sql.Row = [2,244.87138,-5.609,0.37546522455148185,38.898]
song3: org.apache.spark.sql.Row = [3,230.5824,-14.572,0.352224614510411,31.4925]


In [192]:
val msdclusters = msdf6.join(songclusters, msdf6("songID")===songclusters("song_id"))
val cluster1 = msdclusters.filter(msdclusters("Cluster")===song1(0))
val cluster2 = msdclusters.filter(msdclusters("Cluster")===song2(0))
val cluster3 = msdclusters.filter(msdclusters("Cluster")===song3(0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

msdclusters: org.apache.spark.sql.DataFrame = [duration: double, loudness: double ... 5 more fields]
cluster1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [duration: double, loudness: double ... 5 more fields]
cluster2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [duration: double, loudness: double ... 5 more fields]
cluster3: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [duration: double, loudness: double ... 5 more fields]


In [193]:
cluster1.show()
cluster2.show()
cluster3.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------+-------------------+-----------------+------------------+-------+------------------+
| duration|loudness|  artist_hotttnesss|            speed|            songID|Cluster|           song_id|
+---------+--------+-------------------+-----------------+------------------+-------+------------------+
|287.92118|  -6.535| 0.4434068741682247|           28.747|SOYOJKY12AB0185DFD|      0|SOYOJKY12AB0185DFD|
|194.66404|  -8.662|0.48341588921253836|         29.94925|SOGAJHS12A8C13C37B|      0|SOGAJHS12A8C13C37B|
|248.45016|  -5.098|  0.492025942862974|         24.99975|SONWTWU12A8C133E0A|      0|SONWTWU12A8C133E0A|
|263.99302|  -3.695| 0.5180003732384751|           21.252|SOAADAD12A8C13D5B0|      0|SOAADAD12A8C13D5B0|
|222.45832|  -7.867| 0.5981305015736886|            37.72|SOHGFDO12AB017D816|      0|SOHGFDO12AB017D816|
|285.80526|   -8.28|  0.447551326496183|           43.979|SONOJQW12A8C1310BA|      0|SONOJQW12A8C1310BA|
|288.20853|  -7.653| 0.4426392296552482|         31.491

In [194]:
val pred1 = cluster1.select($"song_id", abs($"duration"-song1(1)) + abs($"loudness"-song1(2)) + abs($"artist_hotttnesss"-song1(3)) + abs($"speed"-song1(4)) as "score").orderBy(asc("score")).limit(4)
val pred2 = cluster2.select($"song_id", abs($"duration"-song2(1)) + abs($"loudness"-song2(2)) + abs($"artist_hotttnesss"-song2(3)) + abs($"speed"-song2(4)) as "score").orderBy(asc("score")).limit(4)
val pred3 = cluster3.select($"song_id", abs($"duration"-song3(1)) + abs($"loudness"-song3(2)) + abs($"artist_hotttnesss"-song3(3)) + abs($"speed"-song3(4)) as "score").orderBy(asc("score")).limit(4)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pred1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [song_id: string, score: double]
pred2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [song_id: string, score: double]
pred3: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [song_id: string, score: double]


In [195]:

pred1.join(msdf, pred1("song_id")===msdf("song_id")).select("score", "title").show()
pred2.join(msdf, pred2("song_id")===msdf("song_id")).select("score", "title").show()
pred3.join(msdf, pred3("song_id")===msdf("song_id")).select("score", "title").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+
|             score|               title|
+------------------+--------------------+
|2.8539483974013193|Let's Blame The L...|
|2.3466424772432735|Catchafire (Whoop...|
| 1.356718088949414|   Nails In My Hands|
|               0.0|           Szex & KV|
+------------------+--------------------+

+------------------+--------------------+
|             score|               title|
+------------------+--------------------+
|1.6397829300673092|      No son ariscas|
|               0.0|The Plot to Kill ...|
|1.7822073651316692|   Searched For Love|
|2.0816843435720225|Australian & Anti...|
+------------------+--------------------+

+------------------+--------------------+
|             score|               title|
+------------------+--------------------+
|               0.0|            Prophecy|
| 2.733593793452652|Get Ready (1990 D...|
|1.9609497045910425|           Frasulinu|
|2.1914940013717827|         Yellow Moon|
+------------------+------------